<a href="https://colab.research.google.com/github/jborders23/math6397-funhouse/blob/main/MATH6397_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-Processing

Import & Pre-Process Data

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#load data from GitHub
tesla = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/TSLA.csv")
ford = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/F.csv")
gm = pd.read_csv("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/GM.csv")
sp500 = pd.read_excel("https://raw.githubusercontent.com/jborders23/math6397-funhouse/master/SP500.xlsx")
#reverse sp500 and reset index
sp500 = sp500.iloc[::-1].reset_index(drop=True)


In [12]:
#keep date and closing price for each security
t = tesla.iloc[:,[0,4,7]] #keep 'tomorrow' price for tesla, the target stock
f = ford.iloc[:,[0,4]]
g = gm.iloc[:,[0,4]]
s = sp500.iloc[:,[0,4]]

#add column to tesla that equals 1 if tomorrow>close
t['Movement'] = np.where(t['Tomorrow'] > t['Close'],1,0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
#concatenate dfs
data = pd.concat(objs=[f['Date'],s['Close'],f['Close'],g['Close'],t['Close'],t['Tomorrow'],t['Movement']],axis=1,keys=['Date','SP','Ford','GM','Tesla','Tomorrow','Movement'])
#normalize the time series: xi - min(xi)/max(xi) - min(xi)
s_data = data #will work with a standardized data frame
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
s_data[['SP','Ford','GM','Tesla','Tomorrow']] = scaler.fit_transform(s_data[['SP','Ford','GM','Tesla','Tomorrow']])
s_data.head()

,Date,SP,Ford,GM,Tesla,Tomorrow,Movement
0,4/26/2016,0.041722,0.966270,0.340939,0.017649,0.017113,0
1,4/27/2016,0.043301,0.957341,0.340275,0.017113,0.016226,0
2,4/28/2016,0.034450,1.000000,0.346478,0.016226,0.014585,0
3,4/29/2016,0.029639,0.947421,0.332300,0.014585,0.014831,1
4,5/2/2016,0.037022,0.953373,0.331192,0.014831,0.012593,0


# Convolution

https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/

Prepare Data for Convolution (in progress)

In [16]:
# split s_data into what the network needs
d_data = s_data[['SP','Ford','GM','Tesla','Movement']]
X = d_data.iloc[:,0:4]
y = d_data.iloc[:,-1]

X = np.array(X[:])
X

array([[0.04172216, 0.96626984, 0.3409393 , 0.01764909],
       [0.04330116, 0.95734127, 0.34027472, 0.01711326],
       [0.03444962, 1.        , 0.34647763, 0.01622574],
       ...,
       [0.9768734 , 0.73809524, 0.86331416, 0.80632503],
       [0.99448495, 0.76587302, 0.90141788, 0.83598409],
       [0.97689171, 0.78670635, 0.88303059, 0.8071512 ]])

TypeError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

#build
model = models.Sequential()
model.add(layers.Conv1D(filters=64,kernel_size=10,activation='relu',input_shape=(1,4)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.summary()


